# Setup for single factor experiment


Using only Cardiff only and changing seeds with using LIME and SHAP default values

Will only be done on google colab T4 gpus


* Seed A: 0
* Seed B: 42
* Seed C: 123
* Seed D: 2025

Looking at 1% of the dataset where cardiff_score is 1.00 a quater is TP, TN, FP, FN

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dtale
import matplotlib.pyplot as plt
import seaborn as sns
import re


from master_thesis.config import PROCESSED_DATA_DIR, load_dataframe_from_pickle, save_dataframe_as_pickle

2025-05-06 13:43:25.550 | INFO     | master_thesis.config:<module>:12 - PROJ_ROOT path is: /home/takosaga/Projects/master_thesis


In [2]:
working_df = load_dataframe_from_pickle(
    PROCESSED_DATA_DIR / "models_and_XAI_with_jaccard_similarity_applied.pkl"
)

working_df.head(5)

,text,extracted_target,platform,is_hatespeech,id,cardiff_score,shap_cardiff,lime_cardiff,fb_score,shap_fb,lime_fb,cardiff_hatespeech,fb_hatespeech,lime_cardiff_wordset,lime_fb_wordset,shap_cardiff_wordset,shap_fb_wordset,jaccard_similarity_cardiff,jaccard_similarity_fb
0,i dont think im getting my baby them white 9 h...,{none},twitter,0,1,0.992369,"i : 0.002, getting : 0.001, my : 0.001, baby :...","and: 0.04, my: 0.04, white: -0.03, 9: -0.03",0.687105,": 0.014, i : 0.014, dont : -0.008, think : -0....","white: -0.19, touched: -0.12, dont: -0.11, j: ...",0,1,"{my, and, 9, white}","{touched, j, dont, white}","{them, has, white}","{them, not, white}",0.166667,0.166667
1,we cannot continue calling ourselves feminists...,{none},twitter,0,2,0.997838,"calling : 0.013, ourselves : 0.014, feminists ...","trans: 0.00, gender: 0.00, addressed: 0.00, fe...",0.919145,": -0.014, we : -0.014, cannot : -0.029, contin...","and: 0.13, trans: 0.11, lesbian: 0.11, a: -0.10",0,0,"{addressed, feminists, trans, gender}","{a, and, lesbian, trans}","{ourselves, calling, bisexual, feminists}","{if, sheet, feminists, reporting}",0.142857,0.000000
2,nawt yall niggers ignoring me,{race},twitter,0,3,0.935527,"n: 0.010, aw: 0.006, t : 0.007, y: 0.003, all ...","niggers: -0.90, nawt: -0.09, me: 0.07, yall: 0.07",0.999480,"n: 0.030, aw: -0.062, t : -0.040, y: 0.129, al...","niggers: -0.96, ignoring: -0.02, nawt: -0.02, ...",1,1,"{yall, niggers, nawt, me}","{yall, niggers, ignoring, nawt}","{t, gers, all, ignoring}","{t, gers, all, ignoring}",0.000000,0.142857
3,<user> i am bit confused coz chinese ppl can n...,{race},twitter,1,4,0.997049,": -0.005, <: -0.005, user: -0.005, > : -0.005,...","chong: -0.44, ching: -0.29, chinese: -0.08, pa...",0.779210,": 0.016, <: 0.016, user: 0.016, > : 0.016, i :...","ching: -0.26, this: -0.24, chong: -0.23, confu...",1,1,"{pakistani, ching, chong, chinese}","{confused, ching, this, chong}","{ing, ong, am, inese}","{ing, how, inese, this}",0.000000,0.142857
4,this bitch in whataburger eating a burger with...,"{race, gender}",twitter,1,5,0.990916,"bitch : -0.338, in : -0.145, urger : 0.007, ea...","bitches: -0.44, bitch: -0.28, hate: -0.18, eat...",0.999297,"this : 0.007, bitch : -0.478, in : -0.004, wha...","bitch: -0.39, bitches: -0.30, holding: -0.05, ...",1,1,"{hate, eating, bitches, bitch}","{i, holding, bitches, bitch}","{white, hate, in, bitch}","{white, hate, burger, bitch}",0.333333,0.142857


In [3]:
working_df.shape

(65216, 19)

In [4]:
one_percent = len(working_df)//100

In [5]:
quarter = one_percent//4
quarter

163

In [6]:
working_df.columns.to_list()

['text',
 'extracted_target',
 'platform',
 'is_hatespeech',
 'id',
 'cardiff_score',
 'shap_cardiff',
 'lime_cardiff',
 'fb_score',
 'shap_fb',
 'lime_fb',
 'cardiff_hatespeech',
 'fb_hatespeech',
 'lime_cardiff_wordset',
 'lime_fb_wordset',
 'shap_cardiff_wordset',
 'shap_fb_wordset',
 'jaccard_similarity_cardiff',
 'jaccard_similarity_fb']

In [7]:
columns_to_keep = [
    'text', 
    'extracted_target',
    'platform',
    'is_hatespeech',
    'id',
    'cardiff_score',
    'cardiff_hatespeech'
]

In [8]:
working_df = working_df[columns_to_keep]

In [9]:
working_df.head()

,text,extracted_target,platform,is_hatespeech,id,cardiff_score,cardiff_hatespeech
0,i dont think im getting my baby them white 9 h...,{none},twitter,0,1,0.992369,0
1,we cannot continue calling ourselves feminists...,{none},twitter,0,2,0.997838,0
2,nawt yall niggers ignoring me,{race},twitter,0,3,0.935527,1
3,<user> i am bit confused coz chinese ppl can n...,{race},twitter,1,4,0.997049,1
4,this bitch in whataburger eating a burger with...,"{race, gender}",twitter,1,5,0.990916,1


In [10]:
mask = working_df["cardiff_score"] >= .95
working_df_filtered = working_df[mask]
working_df_filtered.shape

(53395, 7)

In [11]:
# --- Parameters ---
sample_size = 163
ground_truth_col = 'is_hatespeech'
prediction_col = 'cardiff_hatespeech'

# True Positives (TP): is_hatespeech=1, cardiff_hatespeech=1
tp_condition = (working_df_filtered[ground_truth_col] == 1) & (working_df_filtered[prediction_col] == 1)
df_tp = working_df_filtered[tp_condition]

# True Negatives (TN): is_hatespeech=0, cardiff_hatespeech=0
tn_condition = (working_df_filtered[ground_truth_col] == 0) & (working_df_filtered[prediction_col] == 0)
df_tn = working_df_filtered[tn_condition]

# False Positives (FP): is_hatespeech=0, cardiff_hatespeech=1
fp_condition = (working_df_filtered[ground_truth_col] == 0) & (working_df_filtered[prediction_col] == 1)
df_fp = working_df_filtered[fp_condition]

# False Negatives (FN): is_hatespeech=1, cardiff_hatespeech=0
fn_condition = (working_df_filtered[ground_truth_col] == 1) & (working_df_filtered[prediction_col] == 0)
df_fn = working_df_filtered[fn_condition]

In [12]:
print(f"\nNumber of True Positives (TP): {len(df_tp)}")
print(f"Number of True Negatives (TN): {len(df_tn)}")
print(f"Number of False Positives (FP): {len(df_fp)}")
print(f"Number of False Negatives (FN): {len(df_fn)}")


Number of True Positives (TP): 16415
Number of True Negatives (TN): 24929
Number of False Positives (FP): 6516
Number of False Negatives (FN): 5535


In [13]:
# Function to sample and add to list
def sample_category(df_category, category_name, num_samples):
    if len(df_category) >= num_samples:
        sampled_df = df_category.sample(n=num_samples, random_state=42) # random_state for reproducibility
    else:
        print(f"Warning: Category '{category_name}' has only {len(df_category)} samples, which is less than {num_samples}. Taking all available samples.")
        sampled_df = df_category.copy() # Take all if less than desired
    print(f"Sampled {len(sampled_df)} from {category_name}")
    return sampled_df

In [14]:
sampled_dfs = []

sampled_tp = sample_category(df_tp, 'True Positives', quarter)
sampled_tn = sample_category(df_tn, 'True Negatives', quarter)
sampled_fp = sample_category(df_fp, 'False Positives', quarter)
sampled_fn = sample_category(df_fn, 'False Negatives', quarter)

Sampled 163 from True Positives
Sampled 163 from True Negatives
Sampled 163 from False Positives
Sampled 163 from False Negatives


In [15]:
final_sampled_df = pd.concat([sampled_tp, sampled_tn, sampled_fp, sampled_fn], ignore_index=True)
final_sampled_df.shape

(652, 7)

In [16]:
final_sampled_df.head()

,text,extracted_target,platform,is_hatespeech,id,cardiff_score,cardiff_hatespeech
0,"India is one nasty, dirty, filthy, germsy, law...",{nationality},youtube,1,53513,0.994301,1
1,solo que seras bombardeas y destruidos como lo...,{religion},gab,1,8335,0.957913,1
2,@user @user You cannot argue with a retard jer...,"{disability, nationality}",twitter,1,64112,0.994864,1
3,Can I just pound that pussy an pull your hair....,{gender},reddit,1,22034,0.977588,1
4,you do not live in america nazi faggot but you...,{sexuality},gab,1,18143,0.985650,1


In [21]:
save_dataframe_as_pickle(final_sampled_df, PROCESSED_DATA_DIR.as_posix() + '/experiement_samples.pkl')



DataFrame saved to /home/takosaga/Projects/master_thesis/data/processed/experiement_samples.pkl


In [22]:
final_sampled_df.to_csv(PROCESSED_DATA_DIR.as_posix() + '/experiement_samples.csv', index=False)

### Setup for Google Colab

In [ ]:
#%pip install lime shap hf_xet

In [ ]:
# sanity test

from lime.lime_text import LimeTextExplainer
import shap
import numpy as np
import pandas as pd
from transformers import pipeline, AutoTokenizer
import random
import torch
from typing import List, Union
from tqdm.notebook import tqdm 


def set_seed(seed_value):
    """Set seed for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        # Optional: for determinism with CuDNN
        torch.backends.cudnn.deterministic = True



# Set seed for reproducibility
set_seed(42)

# Constants
CLASS_NAMES = ['NOT-HATE', 'HATE']
BATCH_SIZE = 512  # Adjust based on your GPU memory

# Load models and tokenizers once
pipe_cardiff = pipeline("text-classification", 
                       model="cardiffnlp/twitter-roberta-base-hate-latest", 
                       device=0 if torch.cuda.is_available() else -1,
                       batch_size=BATCH_SIZE)

tokenizer_cardiff = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-hate-latest")

# Create dataset class for more efficient processing
class HateSpeechDataset:
    def __init__(self, texts):
        self.texts = texts
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        return self.texts[idx]


# Unified prediction function for both models
def batch_predict(texts: Union[List[str], np.ndarray], pipeline_fn):
    """Run predictions in batches for efficiency"""
    if isinstance(texts, np.ndarray):
        texts = texts.tolist()
        
    dataset = HateSpeechDataset(texts)
    
    # Process in batches
    all_outputs = []
    for i in range(0, len(dataset), BATCH_SIZE):
        batch_texts = [dataset[j] for j in range(i, min(i + BATCH_SIZE, len(dataset)))]
        outputs = pipeline_fn(batch_texts, top_k=2)
        all_outputs.extend(outputs)
    
    # Convert to numpy array with consistent order
    return np.array([
        [label['score'] for label in sorted(res, key=lambda x: x['label'])]
        for res in all_outputs
    ])


# Model-specific prediction functions
def predict_cardiff(texts):
    return batch_predict(texts, pipe_cardiff)



# Initialize explainers
explainer_lime = LimeTextExplainer(class_names=CLASS_NAMES)
explainer_shap_cardiff = shap.Explainer(predict_cardiff, 
                                       masker=shap.maskers.Text(tokenizer_cardiff))

# Helper function for LIME explanations
def lime_explain(text, predictor, num_features=4):
    exp = explainer_lime.explain_instance(text, 
                                          predictor)
    return ", ".join([f"{word}: {weight:.3f}" for word, weight in exp.as_list()])


# Helper function for SHAP explanations
def get_shap_values(text, explainer, class_idx=1):
    shap_vals = explainer([text])
    return ", ".join([
        f"{feature}: {value:.3f}"
        for feature, value in zip(shap_vals[0].data, shap_vals[0].values[:, class_idx])
        if abs(value) > 0.001
    ])


# Sample data
data = {
    'text': [
        "nawt yall niggers ignoring me",
        "and this is why i end up with nigger trainee doctors who can not speak properly lack basic knowledge of biology it truly scary if the public only knew",
        "this is a neutral sentence"
    ]
}
annotated_and_targeted_hatespeech_df = pd.DataFrame(data)


# Process all texts in one batch for efficiency
all_texts = annotated_and_targeted_hatespeech_df["text"].tolist()

# Get predictions in batch
cardiff_preds = pipe_cardiff(all_texts) 

# Process results
results = []
for i, text in enumerate(all_texts):
    try:
        # Get predictions from the batch results
        pred_cardiff = cardiff_preds[i]

        # LIME explanations
        lime_exp_cardiff = lime_explain(text, predict_cardiff)

        # SHAP explanations
        shap_exp_cardiff = get_shap_values(text, explainer_shap_cardiff)

        # Store results
        results.append({
            "text": text,
            "error": None,
            "cardiff_label": pred_cardiff["label"],
            "cardiff_score": pred_cardiff["score"],
            "lime_cardiff": lime_exp_cardiff,
            "shap_cardiff": shap_exp_cardiff
        })
    except Exception as e:
        results.append({
            "text": text,
            "error": f"Caught an exception: {e}",
            "cardiff_label": None,
            "cardiff_score": None,
            "lime_cardiff": None,
            "shap_cardiff": None
        })

final_df = pd.DataFrame(results)


# Print the results
final_df.head()


Device set to use cpu


  0%|          | 0/498 [00:00<?, ?it/s]

,text,error,cardiff_label,cardiff_score,lime_cardiff,shap_cardiff
0,nawt yall niggers ignoring me,None,HATE,0.935527,"niggers: -0.907, nawt: -0.102, me: 0.074, yall...","n: 0.010, aw: 0.006, t : 0.007, y: 0.003, all ..."
1,and this is why i end up with nigger trainee d...,None,HATE,0.993975,"nigger: -0.981, with: -0.007, i: 0.004, doctor...","and : 0.003, this : 0.001, n: -0.007, igger : ..."
2,this is a neutral sentence,None,NOT-HATE,0.999157,"a: 0.001, is: 0.001, this: 0.001, sentence: 0....",


In [18]:
# Saving to drive
from google.colab import drive
drive.mount('/content/drive')

# Save to your Google Drive
final_df.to_csv('/content/drive/MyDrive/hate_speech_analysis_results.csv', index=False)
final_df.to_excel('/content/drive/MyDrive/hate_speech_analysis_results.xlsx', index=False)
final_df.to_pickle('/content/drive/MyDrive/hate_speech_analysis_results.pkl')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import pandas as pd
import requests
from io import BytesIO

# GitHub raw content URL for the pickle file
github_raw_url = "https://github.com/Takosaga/master_thesis/raw/main/data/processed/annotated_and_targeted_hatespeech.pkl" # Updated URL
# Download the pickle file
response = requests.get(github_raw_url)

# Check if the request was successful
if response.status_code == 200:
    # Load the pickle data into a pandas DataFrame
    df = pd.read_pickle(BytesIO(response.content))
    print(f"DataFrame loaded successfully with {len(df)} rows")
    
    # Display the first few rows
    print("\nPreview of the DataFrame:")
    display(df.head())
else:
    print(f"Failed to download the file. Status code: {response.status_code}")
    print(f"Response: {response.text}")

DataFrame loaded successfully with 65217 rows

Preview of the DataFrame:


,id_new,text,extracted_target,label_hatespeech_binary_offensive_not_included,label_hatespeech_binary_offensive_included,label_normal_offensive_hatespeech,platform,original_dataset_title,original_id,original_label,original_target
0,1,i dont think im getting my baby them white 9 h...,{none},not_hatespeech,normal,normal,twitter,HateXplain,1179055004553900032,normal,[None]
1,2,we cannot continue calling ourselves feminists...,{none},not_hatespeech,normal,normal,twitter,HateXplain,1179063826874032128,normal,[None]
2,3,nawt yall niggers ignoring me,{race},not_hatespeech,normal,normal,twitter,HateXplain,1178793830532956161,normal,[African]
3,4,<user> i am bit confused coz chinese ppl can n...,{race},hatespeech,hatespeech/offensive,hatespeech,twitter,HateXplain,1179088797964763136,hatespeech,[Asian]
4,5,this bitch in whataburger eating a burger with...,"{race, gender}",hatespeech,hatespeech/offensive,hatespeech,twitter,HateXplain,1179085312976445440,hatespeech,"[Caucasian, Women]"


2025-05-04 15:40:16,284 - INFO     - Executing shutdown due to inactivity...
2025-05-04 15:40:16,288 - INFO     - Executing shutdown...
2025-05-04 15:40:16,289 - INFO     - Not running with the Werkzeug Server, exiting by searching gc for BaseWSGIServer
/home/takosaga/miniconda3/envs/master_thesis/lib/python3.10/site-packages/dtale/app.py:445: FutureWarning:

`torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead



In [22]:
df[:100].shape

NameError: name 'df' is not defined

In [19]:
def process_dataframe(df, text_column="text"):
    """Process an entire dataframe with efficient batching"""
    all_texts = df[text_column].tolist()
    total_texts = len(all_texts)
    
    # Get predictions in batch
    cardiff_preds = pipe_cardiff(all_texts)
    
    results = []
    for i, text in tqdm(enumerate(all_texts), total=total_texts, desc="Processing texts"):
        try:

            pred_cardiff = cardiff_preds[i]
            
            # Calculate explanations for all examples
            lime_exp_cardiff = lime_explain(text, predict_cardiff)
            shap_exp_cardiff = get_shap_values(text, explainer_shap_cardiff)
            
            results.append({
                "text": text,
                "error": None,
                "cardiff_label": pred_cardiff["label"],
                "cardiff_score": pred_cardiff["score"],
                "lime_cardiff": lime_exp_cardiff,
                "shap_cardiff": shap_exp_cardiff,
            })
        except Exception as e:
            results.append({
                "text": text,
                "error": f"Caught an exception: {e}",
                "cardiff_label": None,
                "cardiff_score": None,
                "lime_cardiff": None,
                "shap_cardiff": None,
            })
    
    return pd.DataFrame(results)

In [ ]:
section = len(df)//1000

In [ ]:
def save_dataframe(df, name, base_path='/content/drive/MyDrive/'):
    """
    Save a dataframe to CSV, Excel, and pickle formats.
    
    Parameters:
    - df: pandas DataFrame to save
    - name: base name for the files (without extension)
    - base_path: directory path where files will be saved
    """
    # Save as CSV
    df.to_csv(f'{base_path}{name}.csv', index=False)
    
    # Save as Excel
    df.to_excel(f'{base_path}{name}.xlsx', index=False)
    
    # Save as pickle
    df.to_pickle(f'{base_path}{name}.pkl')

In [ ]:
set_seed(42)
df_1 = process_dataframe(df[:section])
save_dataframe(df_1, 'df_1_seed42')

In [ ]:
set_seed(0)
df_1 = process_dataframe(df[:section])
save_dataframe(df_1, 'df_1_seed0')